In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [4]:
#Cargo los sets
X_train = pd.read_csv('data/X_train_ohc.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_train = pd.read_csv('data/y_train.csv',header=None,squeeze=True,index_col=0)
X_test = pd.read_csv('data/X_test_ohc.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_test = pd.read_csv('data/y_test.csv',header=None,squeeze=True,index_col=0)

In [5]:
X_submit = pd.read_csv('data/X_submit_ohc.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')

In [6]:
randomForest = RandomForestRegressor(n_estimators=1200, n_jobs= 5, oob_score=True, min_samples_leaf=2, max_features= 0.5)

In [7]:
def RMSE(actual, pred):
    return np.sqrt(mean_squared_error(actual, pred))

In [8]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [9]:
def print_score(r,train_set,test_set,train_target,test_target,F):
    res = [F(r.predict(train_set), train_target),
           F(r.predict(test_set), test_target),
           r.score(train_set, train_target), r.score(test_set, test_target)]
    if hasattr(r, 'oob_score_'): res.append(r.oob_score_)
    print(res)

In [8]:
#Dado que los precios estan acumulados en la izquierda, voy a predecir el logaritmo
#de estos, ya que se parecen a una normal, siendo los datos mas uniformes

features_importancia = pd.DataFrame(randomForest.feature_importances_,\
                                      index = X_train.columns,\
                                      columns=['importancia']).sort_values('importancia', ascending=False)\
                                      

features_importancia = features_importancia[features_importancia['importancia']>0]
features_importancia.head(50)

In [9]:
#Se puede notar una pequeña mejora en las predicciones si se predice el log

In [10]:
#Para despues hacer el submit en su momento hago np.exp(xgb_pred)

Voy a probar hacerlo con los primeros 50 features mas importantes

In [10]:
features_50 = pd.read_csv('data/features_importancia_primera_etapa.csv',index_col=0).head(60)\
            .index.tolist()
features_50

['metroscubiertos',
 'metrostotales',
 'banos',
 'provincia__Distrito Federal',
 'idzona',
 'tipodepropiedad__Apartamento',
 'garages',
 'año',
 'antiguedad',
 'dia',
 'ciudad__Huixquilucan',
 'habitaciones',
 'mes',
 'ciudad__San Pedro Garza García',
 'tipodepropiedad__Terreno',
 'tipodepropiedad__Casa',
 'ciudad__Miguel Hidalgo',
 'provincia__Edo. de México',
 'ciudad__Naucalpan de Juárez',
 'ciudad__Zapopan',
 'ciudad__Benito Juárez',
 'ciudad__Cuauhtémoc',
 'centroscomercialescercanos',
 'escuelascercanas',
 'tipodepropiedad__Terreno comercial',
 'gimnasio',
 'tipodepropiedad__Casa en condominio',
 'tipodepropiedad__Edificio',
 'piscina',
 'ciudad__San Andrés Cholula',
 'provincia__Jalisco',
 'provincia__Yucatán',
 'provincia__Nuevo León',
 'ciudad__Atizapán de Zaragoza',
 'ciudad__Alvaro Obregón',
 'usosmultiples',
 'ciudad__Mérida',
 'ciudad__Iztapalapa',
 'ciudad__Monterrey',
 'ciudad__Tlalpan',
 'ciudad__Coyoacán',
 'tipodepropiedad__Bodega comercial',
 'ciudad__Guadalajara',
 

In [11]:
X_train_50 = X_train[features_50]
X_test_50 = X_test[features_50]

In [12]:
X_train_50.head()

,metroscubiertos,metrostotales,banos,provincia__Distrito Federal,idzona,tipodepropiedad__Apartamento,garages,año,antiguedad,dia,...,ciudad__Cuajimalpa de Morelos,provincia__Querétaro,ciudad__Querétaro,ciudad__San Miguel de Allende,ciudad__Azcapotzalco,tipodepropiedad__Rancho,ciudad__Santa Catarina,ciudad__Gustavo A. Madero,provincia__Morelos,ciudad__Cuernavaca
id,,,,,,,,,,,,,,,,,,,,,
120542,130.0,144.000000,3.0,0,346913.0,0,2.0,2016,8.121536,15,...,0,0,0,0,0,0,0,0,0,0
11955,170.0,130.000000,3.0,0,106912.0,0,1.0,2013,0.000000,19,...,0,0,0,0,0,0,0,0,0,0
38491,64.0,64.000000,1.0,1,23835.0,1,1.0,2014,5.000000,6,...,0,0,0,0,0,0,0,0,0,0
235712,50.0,176.812747,1.0,0,55785.0,0,0.0,2016,8.121536,17,...,0,0,0,0,0,0,0,0,0,0
273251,50.0,50.000000,1.0,1,24517.0,1,1.0,2016,4.000000,28,...,0,0,0,0,0,0,0,0,0,0


In [13]:
%time randomForest.fit(X_train_50, y_train_log)

CPU times: user 28min 23s, sys: 5.95 s, total: 28min 29s
Wall time: 6min 16s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=0.5, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1200, n_jobs=5,
                      oob_score=True, random_state=None, verbose=0,
                      warm_start=False)

In [14]:
%time print_score(randomForest,X_train_50,X_test_50,y_train_log,y_test_log,RMSE)

[0.17112990200527095, 0.3328312062148291, 0.9581769229003299, 0.8430824011357607, 0.842718149493628]
CPU times: user 8min 17s, sys: 1.06 s, total: 8min 18s
Wall time: 1min 41s


In [15]:
#Primer submit
serie_submit = pd.Series(data=np.exp(randomForest.predict(X_submit[features_50])),index=X_submit.index,name='target')

In [16]:
df_entrega = serie_submit.reset_index()
df_entrega.head()

,id,target
0,4941,5.415310e+06
1,51775,9.195222e+05
2,115253,2.402486e+06
3,299321,1.374703e+06
4,173570,5.818813e+05


In [17]:
df_entrega.to_csv('data/submitinicialsinfix.csv',index=False)

In [ ]:
pd.read_csv('data/submit.csv')

In [ ]:
#[0.17161489285239415, 0.3310967160138811, 0.9580282514296621, 0.8436874439242649, 0.8421187086795238]
#el mejor score sin features del log del precio con 1200 estimadores y 60 features